# Пример генерации текста


In [7]:
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

In [2]:
INPUT_FILE = "./data/11-0.txt"
INPUT_FILE = "./data/Dostoevskiy.txt"
# extract the input as a stream of characters
print("Extracting text from input...")
fin = open(INPUT_FILE, 'r', encoding='utf-8')
lines = []
for line in fin:
    line = line.strip().lower()
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

Extracting text from input...


In [3]:
# creating lookup tables
# Here chars is the number of features in our character "vocabulary"
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [4]:
# create inputs and labels from the text. We do this by stepping
# through the text ${step} character at a time, and extracting a 
# sequence of size ${seqlen} and the next output char. For example,
# assuming an input text "The sky was falling", we would get the 
# following sequence of input_chars and label_chars (first 5 only)
#   The sky wa -> s
#   he sky was ->  
#   e sky was  -> f
#    sky was f -> a
#   sky was fa -> l
print("Creating input and label text...")
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


Creating input and label text...


In [5]:
# vectorize the input and label chars
# Each row of the input is represented by seqlen characters, each 
# represented as a 1-hot encoding of size len(char). There are 
# len(input_chars) such rows, so shape(X) is (len(input_chars),
# seqlen, nb_chars).
# Each row of output is a single character, also represented as a
# dense encoding of size len(char). Hence shape(y) is (len(input_chars),
# nb_chars).
print("Vectorizing input and label text...")
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

Vectorizing input and label text...


In [11]:
# Build the model. We use a single RNN with a fully connected layer
# to compute the most likely predicted output char
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
#model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
#                    input_shape=(SEQLEN, nb_chars),
#                    unroll=True))
model.add(LSTM(HIDDEN_SIZE, input_shape=(SEQLEN, nb_chars), dropout=0.2, recurrent_dropout=0.2, return_sequences=False, unroll=True))

model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [12]:
# We train the model in batches and test output generated at each step
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # testing model
    # randomly choose a row from input_chars, then use it to 
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
    print()

Iteration #: 0
Epoch 1/1
1239568/1239568 [==============================] - 1062s - loss: 2.3858  
Generating from seed:  еще довол
 еще довольников подом не посторил с не подом не посторил с не подом не посторил с не подом не посторил с не п
Iteration #: 1
Epoch 1/1
1239568/1239568 [==============================] - 1048s - loss: 2.0833  
Generating from seed: ром тела, 
ром тела, не стало в помнил он принять в том в помнил он принять в том в помнил он принять в том в помнил он п
Iteration #: 2
Epoch 1/1
1239568/1239568 [==============================] - 1070s - loss: 1.9968  
Generating from seed: его взгляд
его взглядел он всё это все дела под бестровить на него просто и просто и просто и просто и просто и просто и 
Iteration #: 3
Epoch 1/1
1239568/1239568 [==============================] - 1048s - loss: 1.9548  
Generating from seed:  не обнару
 не обнарую и при себя на него всё в комнату в комнату в комнату в комнату в комнату в комнату в комнату в ком
Iteration #: 4
Epoch 1/1
123